In [ ]:
!pip install --quiet "langchain" "langchain-community" "beautifulsoup4" "requests" "jinja2" "weasyprint"
!pip install -U --quiet langchain-huggingface
!pip install -q gradio

In [2]:
!pip install --quiet \
    "numpy==1.26.4" \
    "scipy==1.13.1" \
    "scikit-learn==1.5.0" \
    "torch==2.3.1" \
    "torchaudio==2.3.1" \
    "torchvision==0.18.1" \
    "transformers==4.41.2" \
    "accelerate==0.30.1" \
    "bitsandbytes==0.43.1" \
    "diffusers==0.27.2" \
    "huggingface_hub==0.23.2" \
    "peft==0.10.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 91.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 102.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 104.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 27.2 MB/

In [3]:
import os
import gc

import torch
# import torch_xla
# import torch_xla.core.xla_model as xm

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from diffusers import DiffusionPipeline

from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain_huggingface import HuggingFacePipeline
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document

from jinja2 import Environment
from weasyprint import HTML

import gradio as gr

2025-10-01 16:59:06.948226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759337947.149897      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759337947.206947      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
HTML_TEMPLATE_STRING = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{{ title }}</title>
    <style>
        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
            line-height: 1.6;
            color: #333;
            max-width: 800px;
            margin: 20px auto;
            padding: 0 20px;
            background-color: #f9f9f9;
        }
        .container {
            background-color: #ffffff;
            border: 1px solid #ddd;
            border-radius: 8px;
            padding: 40px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.05);
        }
        h1 {
            color: #1a1a1a;
            text-align: center;
            font-size: 2.5em;
            margin-bottom: 0.5em;
        }
        img.header-image {
            max-width: 100%;
            height: auto;
            border-radius: 8px;
            margin-bottom: 20px;
            display: block;
        }
        .divider {
            border: 0;
            height: 1px;
            background: #e0e0e0;
            margin: 30px 0;
        }
        .content-columns {
            display: flex;
            gap: 20px; /* Space between the columns and the vertical line */
        }
        .column {
            flex: 1; /* This makes both columns take up equal space */
            min-width: 0; /* Prevents flexbox overflow */
        }
        .column.left {
            padding-right: 20px;
            border-right: 1px solid #e0e0e0; /* The vertical line */
        }
        p {
            font-size: 1em;
            text-align: justify;
            /* This preserves newlines from the summary text, a cleaner way than replacing \\n */
            white-space: pre-wrap; 
        }
        .footer {
            text-align: center;
            margin-top: 40px;
            font-size: 0.9em;
            color: #888;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>{{ title }}</h1>
        <img src="{{ image_file }}" alt="{{ title }} Header Image" class="header-image">
        <hr class="divider">
        <div class="content-columns">
            <div class="column left">
                <p>{{ summary_left }}</p>
            </div>
            <div class="column right">
                <p>{{ summary_right }}</p>
            </div>
        </div>
    </div>
    <div class="footer">
        <p>Generated by AI Agent</p>
    </div>
</body>
</html>
"""

In [5]:
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# device = xm.xla_device()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"We are now using= {device}")

We are now using= cuda


In [6]:
path_qwen = "/kaggle/input/qwen-3/transformers/14b-base/1"
path_sd = "/kaggle/input/stable-diffusion-xl/pytorch/base-1-0/1"

In [7]:
def scrape_from_web(urls = list, file_path = "content.txt"):
    print("Step 1 initialized, Scraping content from the links")

    all_content = []

    for url in urls:
        try:
            loader = WebBaseLoader(url)
            docs = loader.load()
    
            content = "\n".join([doc.page_content for doc in docs])
            all_content.append(content)
            print(f"Successfully Scraped content from = {url}")
            
        except Exception as e:
            print(f"Error scraping URL= {e}")
            continue

    final_content = "\n\n ++++ARTICLE SEPARATOR++++ \n\n".join(all_content)

    try:
        with open(file_path, 'w', encoding="utf-8") as f:
                f.write(final_content)
    
        print(f"Content saved to {file_path}")
        return final_content
        
    except Exception as e:
        print(f"Error writing to file: {e}")
        return None

In [20]:
def summarize(content_path = "content.txt", summary_path = "summary.txt", title_path = "title.txt", image_prompt_path = "image_prompt.txt"):
    print("\nStep 2 initialized, Summarizing content")

    model = None
    tokenizer = None
    pipe = None
    llm = None
    
    try:
        with open(content_path, 'r', encoding="utf-8") as f:
            content = f.read()

        tokenizer = AutoTokenizer.from_pretrained(path_qwen,
                                                  local_files_only=True,
                                                 )
        model = AutoModelForCausalLM.from_pretrained(path_qwen,
                                                      local_files_only=True,
                                                      device_map="auto",
                                                      torch_dtype="auto"
                                                     )
        
        pipe = pipeline("text-generation", 
                        model = model, 
                        tokenizer = tokenizer,
                        max_new_tokens = 1024,
                        do_sample = True,
                        temperature = 0.8,
                        top_p = 0.95
                       )
        
        # llm = HuggingFacePipeline(pipeline = pipe)

        # text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1024,
        #                                                chunk_overlap = 100
        #                                               )
        # split_docs = text_splitter.create_documents([content])

        # refine_template = (
        #     "Your job is to produce a final summary\n"
        #     "We have provided an existing summary up to a certain point: {existing_answer}\n"
        #     "We have the opportunity to refine the existing summary"
        #     "(only if needed) with some more context below.\n"
        #     "------------\n"
        #     "{text}\n"
        #     "------------\n"
        #     "Given the new context, refine the original summary into a final, high-quality newsletter summary."
        # )
        # refine_prompt = PromptTemplate.from_template(refine_template)

        # question_template = "Summarize the key points of this text for a newsletter:\n\n{text}"
        # question_prompt = PromptTemplate.from_template(question_template)

        # summary_chain = load_summarize_chain(llm = llm, 
        #                                      chain_type = "refine",
        #                                      question_prompt = question_prompt,
        #                                      refine_prompt = refine_prompt,
        #                                      return_intermediate_steps = False
        #                                     )
        
        # summary_clean = summary_chain.invoke(split_docs)['output_text'].strip()

        # Summary part
        summary_prompt = f"Please provide a detailed, well-structured, and comprehensive summary of the following articles. The summary should be written in the style of a professional newsletter. Extract the key insights, main points, and any important conclusions. The final summary should be between 120 and 300 words.\n\nARTICLES:\n\n{content}"
        summary_output = pipe(summary_prompt)
        summary_clean = summary_output[0]['generated_text'][len(summary_prompt):].strip()

        with open(summary_path, 'w', encoding="utf-8") as f:
            f.write(summary_clean)

        print(f"Full Summary saved to {summary_path}")

        # Title part

        print("\nStep 3 initialized, Generating Title")

        title_prompt = f"Based on the following summary, generate one extremely catchy and concise newsletter title. The title must be under 5 words.\n\nSUMMARY:\n{summary_clean}"
        title_output = pipe(title_prompt, max_new_tokens=20)
        final_title = title_output[0]['generated_text'][len(title_prompt):].strip().split('\n')[0].strip().lstrip('1234567890.-* ')
        
        with open(title_path, 'w', encoding="utf-8") as f: 
            f.write(final_title)
            
        print(f"Generated title saved to {title_path}")
        
        
        # Image prompt part

        print("\nStep 4 initialized, Generating Image prompt")

        image_gen_prompt = f"Based on the following text, create a short, visually descriptive prompt for an AI image generator. Focus on concrete objects, scenes, colors, and styles. Do not include abstract concepts. The prompt should be a single, concise sentence of no more than 50 words.\n\nTEXT:\n{summary_clean}"
        image_prompt_output = pipe(image_gen_prompt, max_new_tokens=60)
        final_image_prompt = image_prompt_output[0]['generated_text'][len(image_gen_prompt):].strip().replace('"', '')
        
        with open(image_prompt_path, "w", encoding="utf-8") as f: 
            f.write(final_image_prompt)
            
        print(f"Generated prompt saved to {image_prompt_path}")

        return summary_clean, final_title, final_image_prompt

    finally:
        print("Cleaning up Qwen model from memory")

        if 'model' in locals() and model is not None:
            del model
        if 'tokenizer' in locals() and tokenizer is not None:
            del tokenizer
        if 'pipe' in locals() and pipe is not None:
            del pipe
        if 'llm' in locals() and llm is not None:
            del llm
        clear_memory()

In [21]:
# def generate_title_and_prompt(summary_path = "summary.txt", title_path = "title.txt", image_prompt_path = "image_prompt.txt"):
#     print("Step 3 initialized. Generating title")

#     gen_model = None
#     gen_tokenizer = None
#     gen_pipe = None
    
#     try:
# #Title Part
#         with open(summary_path, 'r', encoding="utf-8") as f:
#             summary_clean = f.read()

#         gen_tokenizer = AutoTokenizer.from_pretrained(path_phi3,
#                                                       local_files_only=True,
#                                                       trust_remote_code=True)
#         gen_model = AutoModelForCausalLM.from_pretrained(path_phi3,
#                                                          local_files_only=True,
#                                                          trust_remote_code=True, 
#                                                          device_map="auto", 
#                                                          torch_dtype="auto", 
#                                                          load_in_8bit=(device == "cuda")
#                                                         )
#         gen_pipe = pipeline("text-generation", 
#                         model = gen_model, 
#                         tokenizer = gen_tokenizer,
#                         max_new_tokens = 80,
#                         eos_token_id = gen_tokenizer.eos_token_id
#                        )

#         title_messages = [
#             {"role": "user", "content": f"Based on this summary, generate only one extremely catchy but concise newsletter title. The title must be under 12 words.\n\nSUMMARY:\n{summary_clean}"}
#         ]

#         title_outputs = gen_pipe(title_messages, max_new_tokens = 30)

#         generated_title = title_outputs[0]['generated_text'][-1]['content']
#         final_title = generated_title.strip().split('\n')[0].strip().lstrip('1234567890.-* ')
        
#         with open(title_path, 'w', encoding="utf-8") as f:
#             f.write(final_title)

#         print(f"title saved to {title_path}")

# #Image prompt Part        
#         print("Step 4 initialized. Generating image prompt")

#         prompt_messages = [{"role": "user", "content": f"Based on the following text, create a short, visually descriptive prompt for an AI image generator. Focus on concrete objects, scenes, colors, and styles. Do not include abstract concepts. Maximum 60 words.\n\nTEXT:\n{summary_clean}"}]
#         prompt_outputs = gen_pipe(prompt_messages, max_new_tokens=80)
#         generated_prompt = prompt_outputs[0]['generated_text'][-1]['content']
#         final_image_prompt = generated_prompt.strip().replace('"', '')
        
#         with open(image_prompt_path, "w", encoding="utf-8") as f:
#             f.write(final_image_prompt) 

#         print(f"image prompt saved to {image_prompt_path}")

#         return final_title, final_image_prompt
#     finally:
#         print("Cleaning up title gen model from memory")
#         del gen_model
#         del gen_tokenizer
#         del gen_pipe
#         clear_memory()

In [22]:
def generate_newsletter_image(image_prompt_path = "image_prompt.txt", title_path = "title.txt", image_path = "newsletter_image.png"):
    print("\nStep 5 initialized, Generating image for newsletter")

    pipe = None
    
    try:
        with open(image_prompt_path, 'r', encoding="utf-8") as f:
            visual_prompt = f.read().strip()
        with open(title_path, 'r', encoding="utf-8") as f:
            title = f.read().strip()

        final_prompt = f"Image for a newsletter titled '{title}'. A clean, professional illustration of: {visual_prompt}. Marketing aesthetic, vibrant colors."
        
        pipe = DiffusionPipeline.from_pretrained(
            path_sd,
            torch_dtype = torch.float16,
            variant = "fp16",
            use_safetensors = True
        )

        pipe = pipe.to(device)

        image = pipe(prompt = final_prompt).images[0]
        image.save(image_path)
        print(f"Image saved to {image_path}")

        return image_path

    finally:
        print("Cleaning up diffusion model from memory")
        
        if 'pipe' in locals() and pipe is not None:
            del pipe

        clear_memory()

In [23]:
def assemble(summary_path, image_path, title_path, output_name="newsletter"):
    print("Step 6 initialized, Assembling final Newsletter")

    try:
        with open(summary_path, 'r', encoding="utf-8") as f:
            summary_text = f.read()
        with open(title_path, 'r', encoding="utf-8") as f:
            title = f.read().strip()

        mid = len(summary_text) // 2
        split_point = summary_text.find(' ', mid)

        if split_point == -1:
            split_point = mid

        summary_left = summary_text[:split_point]
        summary_right = summary_text[split_point:].strip()

        template = Environment().from_string(HTML_TEMPLATE_STRING)

        html_content = template.render(
            title = title,
            summary_left = summary_left,
            summary_right = summary_right,
            image_file = image_path
        )

        html_file_path = f"{output_name}.html"

        with open(html_file_path, 'w', encoding="utf-8") as f:
            f.write(html_content)

        print(f"HTML Newsletter saved to {html_file_path}")

        return html_file_path
        
    except Exception as e:
        print(f"An error occured during assembly= {e}")

In [24]:
def run_full_pipeline(urls_string):
    print("Starting automated Newsletter Generation")

    urls = [url.strip() for url in urls_string.split(',') if url.strip()]
    if not urls:
        yield "Please upload atleast one url", None
        return
    
    content_file = "content.txt"
    summary_file = "summary.txt"
    title_file = "title.txt"
    image_prompt_file = "image_prompt.txt"
    image_file = "newsletter_image.png"

    log = ""
    try:
        log += "Step 1/4= Scraping content from URLs+++++\n"
        yield log, None
        scrape_from_web(urls, content_file)
        log += "Step 1/4= Scraping Completed\n"
        yield log, None

        log += "Step 2/4= Summarizing, Generating Title+Image Prompt Content+++++\n"
        yield log, None
        summarize(content_file, summary_file, title_file, image_prompt_file)
        log += "Step 2/4= Summarizing, Generating Title+Image Prompt Complete\n"
        yield log, None

        # yield "Step 3/5= Generating Title and Image_prompt+++++", None
        # generate_title_and_prompt(summary_file, title_file, image_prompt_file)
        # yield "Step 3/5= Title and Image_prompt Generated", None
        
        log += "Step 3/4= Generating Image++++\n"
        yield log, None
        image_file_path = generate_newsletter_image(image_prompt_file, title_file, image_file)
        log += "Step 3/4= Image Generated\n"
        yield log, None

        
        log += "Step 4/4= Assembling final Newsletter+++++\n"
        yield log, None
        html_file_path = assemble(summary_file, image_file_path, title_file)
        log += "Step 4/4= Newsletter Assembled\n"
        yield log, None


        log += "Pipeline finished!!! Your newsletter is ready for download."    

        downloadables = [content_file, summary_file, title_file, image_prompt_file, image_file_path, html_file_path]

        yield log, downloadables
        
    except Exception as e:
        error_message = f"Error occurred = {str(e)}"
        print(error_message)
        yield error_message, None

In [ ]:
with gr.Blocks(theme='monochrome', title="Agentic Newsletter Generator") as demo:
    gr.Markdown('''<div style="position: absolute; top: 20px; right: 20px; opacity: 0.9; z-index: -1;">
  <img src="https://ibb.co/jkfszwYT" alt="Watermark" width="200">
</div>

<center>
<h1>
Agentic Newsletter Generator
</h1>
</center>

- Enter one or more URLs (comma+separated)
- The agent will automatically parse, summarize, and newsletter will be created
---------
''')
    
    with gr.Row():
        with gr.Column(scale = 2):
            url_input = gr.Textbox(label="Enter URLs Here", placeholder="https://www.<websitename>.<domain>/")
            generate_button = gr.Button("Generate Newsletter", variant="primary")

        with gr.Column(scale = 1):
            status_output = gr.Textbox(label="Agent's progress", interactive=False, lines=8)
            file_output = gr.File(label="Downloadables output")

    generate_button.click(
        fn = run_full_pipeline,
        inputs = url_input,
        outputs=[status_output, file_output]
    )

if __name__ == "__main__":
    demo.launch(debug=True, share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://83b3b258ca684063a8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Starting automated Newsletter Generation
Step 1 initialized, Scraping content from the links
Successfully Scraped content from = https://www.theverge.com/24172762/sonos-ace-headphones-video-review
Content saved to content.txt

Step 2 initialized, Summarizing content


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at /kaggle/input/pegasus-daily-local/hub/google--pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Your max_length is set to 512, but your input_length is only 217. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=108)
Your max_length is set to 512, but your input_length is only 458. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=229)
Your max_length is set to 512, but your input_length is only 472. Since this is a summarization task, where outpu

Full Summary saved to summary.txt
Cleaning up summarization model from memory
Step 3 initialized. Generating title


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

title saved to title.txt
Step 4 initialized. Generating image prompt
image prompt saved to image_prompt.txt
Cleaning up title gen model from memory

Step 5 initialized, Generating image for newsletter


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (94 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ["google's ai search restrictions and vox media's copyright.. marketing aesthetic, vibrant colors."]
Token indices sequence length is longer than the specified maximum sequence length for this model (94 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ["google's ai search restrictions and vox media's copyright.. marketing aesthetic, vibrant colors."]


  0%|          | 0/50 [00:00<?, ?it/s]

Image saved to newsletter_image.png
Cleaning up diffusion model from memory
Step 4 initialized, Assembling final Newsletter
HTML Newsletter saved to newsletter.html
